In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")

pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 100)
from tqdm import tqdm_notebook
from sklearn.preprocessing  import LabelEncoder

import lightgbm as lgb
import time
import datetime
from sklearn.model_selection import StratifiedKFold, KFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#https://www.kaggle.com/theoviel/load-the-totality-of-the-data
dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float32',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int16',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float64', # was 'float32'
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float32', # was 'float16'
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float32', # was 'float16'
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float64', # was 'float32'
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float64', # was 'float32'
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32', # was 'float16'
        'Census_InternalPrimaryDisplayResolutionVertical':      'float32', # was 'float16'
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float64', # was 'float32'
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
%%time
train = pd.read_csv('/kaggle/input/train.csv',  dtype= dtypes ,nrows = 2000000)
test = pd.read_csv('/kaggle/input/test.csv' , dtype = dtypes )
train  = reduce_mem_usage(train)
test  = reduce_mem_usage(test)
score = []

In [ ]:
print('Number of columns in train data is ->',train.shape[1])
float_list = []
int_list = []
category_list = []
for f in train.columns:
    if train[f].dtype =='int64' or train[f].dtype =='int8' or train[f].dtype =='int16' or train[f].dtype =='int32':
        int_list.append(f)
    elif train[f].dtype =='float64' or train[f].dtype =='float32' or train[f].dtype =='float16':
        float_list.append(f)
    else:
        category_list.append(f)
print('Number of columns in train data of Float Type is ->', len(float_list))
print('Number of columns in train data of Int Type is ->', len(int_list))
print('Number of columns in train data of Category Type is ->', len(category_list))

In [ ]:
print(train['HasDetections'].value_counts()/train.shape[0]*100)

In [ ]:
print('Creating data info of train data and test data .. ...')
stats = []
for f in train.columns:
    stats.append((f ,train[f].nunique() , train[f].isnull().sum()*100/train.shape[0] , 
                  train[f].value_counts(normalize = True , dropna = True).values[0]*100 , 
                  train[f].value_counts(normalize = True , dropna = False).values[0]*100 ,
                  train[f].dtype))
stats_df = pd.DataFrame(stats , columns= ['Columns', 'Number  of Unique Values' , 'Percentage of Data missing in train' ,
                                          'Percentage of Biggest Category of dropna True' , 
                                          'Percentage of Biggest Category of dropna False' ,
                                          'Columns DataType'])
display(stats_df.sort_values('Percentage of Data missing in train', ascending=False))

In [ ]:
col_temp = ['EngineVersion','AppVersion','AvSigVersion', 'OsBuildLab' , 'OsVer' , 'Census_OSVersion']
for f in col_temp:
    print('Null Values in column ',f,' is ->',train[f].isnull().sum())

In [ ]:
def add_category_1(data):
    data['EngineVersion_0'] = data['EngineVersion'].apply(lambda x : x.split('.')[0]).astype('category')
    data['EngineVersion_1'] = data['EngineVersion'].apply(lambda x : x.split('.')[1]).astype('category')
    data['EngineVersion_2'] = data['EngineVersion'].apply(lambda x : x.split('.')[2]).astype('category')
    data['EngineVersion_3'] = data['EngineVersion'].apply(lambda x : x.split('.')[3]).astype('category')
    data['AppVersion_0'] = data['AppVersion'].apply(lambda x : x.split('.')[0]).astype('category')
    data['AppVersion_1'] = data['AppVersion'].apply(lambda x : x.split('.')[1]).astype('category')
    data['AppVersion_2'] = data['AppVersion'].apply(lambda x : x.split('.')[2]).astype('category')
    data['AppVersion_3'] = data['AppVersion'].apply(lambda x : x.split('.')[3]).astype('category')
    data['AvSigVersion_0'] = data['AvSigVersion'].apply(lambda x : x.split('.')[0]).astype('category')
    data['AvSigVersion_1'] = data['AvSigVersion'].apply(lambda x : x.split('.')[1]).astype('category')
    data['AvSigVersion_2'] = data['AvSigVersion'].apply(lambda x : x.split('.')[2]).astype('category')
    data['AvSigVersion_3'] = data['AvSigVersion'].apply(lambda x : x.split('.')[3]).astype('category')
    data['OsBuildLab_0'] = data['OsBuildLab'].apply(lambda x : x.split('.')[0]).astype('category')
    data['OsBuildLab_1'] = data['OsBuildLab'].apply(lambda x : x.split('.')[1]).astype('category')
    data['OsBuildLab_2'] = data['OsBuildLab'].apply(lambda x : x.split('.')[2]).astype('category')
    data['OsBuildLab_3'] = data['OsBuildLab'].apply(lambda x : x.split('.')[3]).astype('category')
    data['OsVer_0'] = data['OsVer'].apply(lambda x : x.split('.')[0]).astype('category')
    data['OsVer_1'] = data['OsVer'].apply(lambda x : x.split('.')[1]).astype('category')
    data['OsVer_2'] = data['OsVer'].apply(lambda x : x.split('.')[2]).astype('category')
    data['OsVer_3'] = data['OsVer'].apply(lambda x : x.split('.')[3]).astype('category')
    data['Census_OSVersion_0'] = data['Census_OSVersion'].apply(lambda x : x.split('.')[0]).astype('category')
    data['Census_OSVersion_1'] = data['Census_OSVersion'].apply(lambda x : x.split('.')[1]).astype('category')
    data['Census_OSVersion_2'] = data['Census_OSVersion'].apply(lambda x : x.split('.')[2]).astype('category')
    data['Census_OSVersion_3'] = data['Census_OSVersion'].apply(lambda x : x.split('.')[3]).astype('category')
    return data

In [ ]:
print('Earlier Shape of train data is ', train.shape)
train = add_category_1(train)
test = add_category_1(test)
print('New Shape of train is ' , train.shape)

In [ ]:
# Remove Columns list
remove_list = []
# Removing  from Null Columns
for f in train.columns:
    if train[f].isnull().sum()*100/train.shape[0] > 98 :
        remove_list.append(f)
# Removing the data which has high Bias 
for f in train.columns:
    if train[f].value_counts().values[0]*100/train.shape[0] > 98:
        remove_list.append(f)
print(remove_list)

In [ ]:
train.drop(columns = remove_list , inplace = True)
test.drop(columns = remove_list , inplace = True)

In [ ]:
cat_col = []
for f in train.columns:
    if str(train[f].dtype)== 'category':
        cat_col.append(f)
print(len(cat_col))
print(cat_col)

In [ ]:
to_encode = []
for f in cat_col:
    if train[f].nunique() >100:
        to_encode.append(f)
print(to_encode)

In [ ]:
to_encode.remove('MachineIdentifier')
cat_col.remove('MachineIdentifier')

In [ ]:
def frequency_encoder(variable):
    df  = pd.concat([ train[variable]  , test[variable] ]).value_counts().reset_index()
#     df  = test[variable].value_counts().reset_index()
    # df[to_encode[1]].value_counts().reset_index().reset_index().rename(columns = {'level_0': 'rank_encode' , 'index':to_encode[1] , to_encode[1]:'count'})
    df = df.merge(df[variable].value_counts().reset_index().reset_index().rename(columns = {'level_0': 'rank_encode' , 'index':variable , variable:'count'}) , on =variable , how='left' )[['index' , 'rank_encode']]
    df.set_index('index' , inplace = True)
    return df.to_dict()['rank_encode']

In [ ]:
%%time
for f in tqdm_notebook(to_encode):
    dictionary = frequency_encoder(f)
    train[f+'_ranking'] = train[f].apply(lambda x: dictionary.get(x , np.nan)).astype('category')
    test[f+'_ranking'] = test[f].apply(lambda x : dictionary.get(x , np.nan)).astype('category')


In [ ]:
%%time
for f in tqdm_notebook(cat_col):
#     print('Working on -> ',f)
    lb = LabelEncoder()
    lb.fit(list(train[f].values) + list(test[f].values))
    train[f] = lb.transform(list(train[f].values))
    test[f] = lb.transform(list(test[f].values))

In [ ]:
train.head()

In [ ]:
y = train['HasDetections']
train.drop(columns=['HasDetections' , 'MachineIdentifier' ] , inplace = True)
# train.drop(columns = ['Census_IsFlightingInternal',
#  'Census_IsWIMBootEnabled', 'Census_IsAlwaysOnAlwaysConnectedCapable', 'Census_IsPenCapable', 'OsPlatformSubRelease',
#  'Census_OSBuildNumber', 'Census_OSArchitecture', 'OsBuildLab_2', 'OsBuildLab_0',
#  'Census_ProcessorManufacturerIdentifier', 'Census_OSVersion_2', 'Platform', 'OsVer', 'OsVer_0', 'OsVer_1'] , inplace = True)
test_index = list(test['MachineIdentifier'])
test.drop(columns = ['MachineIdentifier'] , inplace = True)
train.sort_values('AvSigVersion')
n_folds = 3
folds = StratifiedKFold(n_splits = n_folds , shuffle  = True , random_state = 98)


In [ ]:
%%time
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

In [ ]:
for  i,j in zip(train.columns , test.columns):
    if i != j:
        print('Possible Mismatch ' ,i , j)
    if str(train[i].dtype) != str(test[j].dtype):
        print(i ,',',j)

In [ ]:
params = {'num_leaves': 256,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 5,
         'learning_rate': 0.05,
         "boosting": "gbdt",
         "feature_fraction": 0.8,
         "bagging_freq": 5,
         "bagging_fraction": 0.8,
         "bagging_seed": 11,
         "lambda_l1": 0.15,
         "lambda_l2": 0.15,
         "random_state": 42,          
         "verbosity": -1}

In [ ]:
test_1 = test[ : int(test.shape[0]/3)]
test_2 = test[int(test.shape[0]/3) : int(test.shape[0]/3*2)]
test_3 = test[int(test.shape[0]/3*2) : ]

In [ ]:
del test
gc.collect()

In [ ]:
# from numba import jit

def fast_auc(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc

def eval_auc(preds, dtrain):
#     labels = dtrain.get_label()
    return fast_auc(dtrain, preds)
train_col = train.columns
feature_importance = pd.DataFrame()
pred_1 = np.zeros(test_1.shape[0])
pred_2 = np.zeros(test_2.shape[0])
pred_3 = np.zeros(test_3.shape[0])
feature_importance['columns'] = train.columns
for n_fold , (train_index , valid_index) in enumerate(folds.split(train , y)):
    gc.collect()
    print('Fold ',n_fold , ' Started at ',time.ctime())
    X_train , X_valid = train.iloc[train_index] , train.iloc[valid_index]
    y_train , y_valid = y.iloc[train_index] , y.iloc[valid_index]
    train_data = lgb.Dataset(X_train[train_col] , label = y_train )
    valid_data = lgb.Dataset(X_valid[train_col] , label = y_valid )
    model = lgb.train(params , train_data , num_boost_round = 1000 , valid_sets = [train_data , valid_data] , verbose_eval =-1, early_stopping_rounds = 200 )
    pred = model.predict(X_valid ,  num_iteration=model.best_iteration)
    pred = list(pred)
    pred_1+=model.predict(test_1[train_col])/n_folds
    pred_2+=model.predict(test_2[train_col])/n_folds
    pred_3+=model.predict(test_3[train_col])/n_folds
    score.append((r2_score(pred , y_valid) , mean_squared_error(pred , y_valid) , eval_auc(pred , y_valid)))
    feature_importance['fold_'+str(n_fold+1)] = model.feature_importance()
    

In [ ]:
print('Shape of each pred is ',pred_1.shape,pred_2.shape,pred_3.shape)
pred = np.concatenate((pred_1 , pred_2 , pred_3))
del pred_1 , pred_2 , pred_3
gc.collect()
print(pred[:20])

In [ ]:
display(score)
feature_importance['avg'] = (feature_importance['fold_1']+feature_importance['fold_2']+feature_importance['fold_3'])/3
feature_importance.head()


In [ ]:
plt.figure(figsize=(16, 26));
plt.subplot(5,1,1)
sns.barplot(x="fold_1", y="columns", data=feature_importance.sort_values(by="fold_1", ascending=False).head(15))
plt.title('Feature Importance for fold_1')
plt.subplot(5,1,2)
sns.barplot(x="fold_2", y="columns", data=feature_importance.sort_values(by="fold_2", ascending=False).head(15))
plt.title('Feature Importance for fold_2')
plt.subplot(5,1,3)
sns.barplot(x="fold_3", y="columns", data=feature_importance.sort_values(by="fold_3", ascending=False).head(15))
plt.title('Feature Importance for fold_3')
plt.subplot(5,1,4)
sns.barplot(x="avg", y="columns", data=feature_importance.sort_values(by="avg", ascending=False).head(15))
plt.title('Feature Importance for avg')
plt.subplot(5,1,5)
sns.barplot(x="avg", y="columns", data=feature_importance.sort_values(by="avg", ascending=False).tail(15))
plt.title('Feature Importance for avg tail')
plt.plot()

In [ ]:
# to_remove = list(feature_importance.sort_values(by="avg", ascending=False).tail(15)['columns'])

In [ ]:
submission = pd.DataFrame()
submission['MachineIdentifier'] = test_index
submission['HasDetections'] = pred
submission.to_csv('lgb_first.csv' , index = False)

In [ ]:
submission.sample(10)

* The First Submission with 3rd commit gives score of Private Score 0.63435 , Public Score 0.67810
* Second submission with 5th commit gives score of Private Score 0.60323 Public Score 0.66492